In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

In [3]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)), #Conv2D層
            #過濾器(卷積核)數量, 過濾器(卷積核)長寬, input_shape=(image_height, image_weight, image_channels) 此處不包含批次維度
    layers.MaxPooling2D((2, 2)), #MaxPooling2D層
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu')
])

2022-08-12 13:25:54.980954: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [4]:
model.summary()
#Param(參數) = 過濾器長 * 寬 * 通道數 * 過濾器個數 + 過濾器個數 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [5]:
#在卷積神經網路加上分類器
#最後一個Conv2D層輸出的3D張量(shape=(3,3,64))，須送至分類器神經網路(Dense密集層)，但僅能處理1D張量，因此需將其展平
model.add(layers.Flatten()) #3D張量展平為1D張量，shape=(576,)
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax')) #具有10個輸出的輸出層

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [7]:
#用MNIST影像訓練卷積神經網路
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical

In [8]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [9]:
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

In [10]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels) 

In [11]:
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [12]:
model.fit(train_images, train_labels,
          epochs=5,
          batch_size=64)

2022-08-12 13:25:56.475104: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
938/938 [==============================] - 21s 22ms/step - loss: 0.1749 - accuracy: 0.9441
Epoch 2/5
938/938 [==============================] - 20s 22ms/step - loss: 0.0456 - accuracy: 0.9863
Epoch 3/5
938/938 [==============================] - 21s 22ms/step - loss: 0.0319 - accuracy: 0.9905
Epoch 4/5
938/938 [==============================] - 20s 21ms/step - loss: 0.0241 - accuracy: 0.9930
Epoch 5/5
938/938 [==============================] - 22s 23ms/step - loss: 0.0190 - accuracy: 0.9943


In [13]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

313/313 [==============================] - 2s 6ms/step - loss: 0.0346 - accuracy: 0.9897


In [14]:
test_acc

0.9897000193595886